In [1]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('/home/jupyter-23521027/refresh-bert/data/sbert/indosbert_dense_trained', device='cuda:1')

In [19]:
import os
import json
import pandas as pd

folder_path = '/home/jupyter-23521027/refresh-bert/data/liputan6_data/canonical/train' 

file_list = os.listdir(folder_path)
json_data_list = []
file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name) 

    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            json_data_list.append(json_data)

df = pd.DataFrame(json_data_list)
df['sentences'] = df['clean_article'].apply(lambda x: [' '.join(sentence) for sentence in x])

,id,url,clean_article,clean_summary,extractive_summary,sentences
193882,65953,https://www.liputan6.com/news/read/65953/ditem...,"[[Liputan6, ., com, ,, Langkat, :, Tim, Search...","[[Tim, SAR, kembali, menemukan, satu, jenazah,...","[1, 2, 3]","[Liputan6 . com , Langkat : Tim Search and Res..."


In [20]:
df.tail(5)

,id,url,clean_article,clean_summary,extractive_summary,sentences
193878,164143,https://www.liputan6.com/news/read/164143/tahu...,"[[Liputan6, ., com, ,, Jakarta, :, Warga, yang...","[[Dirjen, Pajak, membuat, aturan, setiap, oran...","[2, 4]","[Liputan6 . com , Jakarta : Warga yang belum m..."
193879,150247,https://www.liputan6.com/news/read/150247/mass...,"[[Liputan6, ., com, ,, Bogor, :, Ribuan, umat,...","[[Ribuan, umat, Islam, menyerbu, vila, milik, ...","[0, 1]","[Liputan6 . com , Bogor : Ribuan umat Islam me..."
193880,204203,https://www.liputan6.com/news/read/204203/bate...,"[[Chelsea, melayangkan, keberatannya, kepada, ...","[[Ketua, Leeds, United, ,, Ken, Bates, berhasi...","[0, 13]",[Chelsea melayangkan keberatannya kepada Asosi...
193881,159188,https://www.liputan6.com/news/read/159188/peny...,"[[Liputan6, ., com, ,, Denpasar, :, Kebakaran,...","[[Kebakaran, toko, sepatu, di, Denpasar, ,, Ba...","[0, 3]","[Liputan6 . com , Denpasar : Kebakaran toko se..."
193882,65953,https://www.liputan6.com/news/read/65953/ditem...,"[[Liputan6, ., com, ,, Langkat, :, Tim, Search...","[[Tim, SAR, kembali, menemukan, satu, jenazah,...","[1, 2, 3]","[Liputan6 . com , Langkat : Tim Search and Res..."


In [14]:
file_path = "/home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.training.sbert"

existing_ids = []
with open(file_path, 'r') as file:
    lines = file.readlines()

    for line in lines:
        line = line.strip()

        if line and ' ' not in line:
            existing_ids.append(line.split("-")[1])
            
print(existing_ids[:5])
print(len(existing_ids))

['48951', '124973', '298677', '40569', '242869']
193883


In [16]:
print(existing_ids[-5:])


['164143', '150247', '204203', '159188', '65953']


In [5]:
from datetime import datetime

MAX_DOC_LENGTH = 100
MAX_SENT_EMBEDDING_LENGTH = 250
DATASET_SIZE = 999999

now = datetime.now()
file = open(file_path , 'a')
file.write("\n")

for i, row in df.iterrows():
    if str(row['id']) in existing_ids:
        continue

    file.write(f"liputan6-{str(row['id'])}\n")

    for sent_txt in row["sentences"][:MAX_DOC_LENGTH]:
        sent_embedding = model.encode(sent_txt, convert_to_numpy=True).tolist()[:MAX_SENT_EMBEDDING_LENGTH]

        for element in sent_embedding:
            file.write(str(element) + " ")
            
        file.write("\n")
    
    if (i + 1) % 100 == 0:
        print(f"processing {i + 1} with time: {datetime.now() - now}")
        now = datetime.now()
    
    if (i + 1) == DATASET_SIZE:
        break
    
    file.write("\n")
        
file.close()

processing 5100 with time: 0:00:23.636837
processing 5200 with time: 0:00:21.123895
processing 5300 with time: 0:00:18.692735
processing 5400 with time: 0:00:21.061198
processing 5500 with time: 0:00:19.007029
processing 5600 with time: 0:00:20.055462
processing 5700 with time: 0:00:18.904964
processing 5800 with time: 0:00:21.010219
processing 5900 with time: 0:00:18.675026
processing 6000 with time: 0:00:19.189577
processing 6100 with time: 0:00:23.863566
processing 6200 with time: 0:00:21.583303
processing 6300 with time: 0:00:24.803183
processing 6400 with time: 0:00:23.646239
processing 6500 with time: 0:00:22.975523
processing 6600 with time: 0:00:20.490334
processing 6700 with time: 0:00:23.905339
processing 6800 with time: 0:00:25.282988
processing 6900 with time: 0:00:23.601719
processing 7000 with time: 0:00:22.541612
processing 7100 with time: 0:00:27.213090
processing 7200 with time: 0:00:22.028409
processing 7300 with time: 0:00:22.722354
processing 7400 with time: 0:00:25

In [32]:
!cat /home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.test.sbert | tail -c 100000

6155281067 -0.6358654499053955 -0.783704400062561 -0.4321339428424835 0.5925198197364807 0.2758817672729492 0.28675344586372375 0.7413437962532043 -0.696352481842041 -0.10373382270336151 0.6342480182647705 0.4117718040943146 0.7423290014266968 -0.7732146978378296 -0.005164458882063627 0.403552770614624 -0.4940550923347473 -0.36400243639945984 0.5322866439819336 0.20016241073608398 
-0.1965930014848709 0.37839654088020325 0.5285086035728455 -0.6398188471794128 0.12636105716228485 0.3985401391983032 -0.5121103525161743 0.005570360459387302 -0.7933135032653809 0.19525685906410217 0.2608638107776642 0.23389120399951935 -0.011537158861756325 -0.26709115505218506 -0.49080777168273926 -0.728370726108551 0.49788081645965576 -0.0346028134226799 -0.6228067278862 0.766058623790741 -0.6653158068656921 -0.8357224464416504 0.5956425666809082 0.268420934677124 -0.15601691603660583 0.749914288520813 0.6132984757423401 0.5269899368286133 0.46801766753196716 -0.7117889523506165 -0.19452005624771118 -0.5